In [1]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning1 as fsl

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

/var/folders/wp/4cr2jl295bs4z4czxy_p6w5m0000gn/T/ipykernel_94175/735377492.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/mathiasperez/Documents/GitHub/442-2-Protein-cleavage/fonctionsSupervisedLearning1.py:124: SyntaxWarning: invalid escape sequence '\ '
  '''


In [2]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')

In [3]:
##############################
## RECUPERATION DES DONNÉES ##
##############################


# Process each entry
processed_entries = [fsl.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [4]:
df_exploitable = fsl.convert_df_to_vectors(df)

In [5]:
n = 17  # length of the subsequence
svm_model_in = svm.SVC(kernel='rbf', C=1, random_state=42)

svm_model_pos = svm.SVC(kernel='linear', C=1, random_state=42)




Train

In [11]:
svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(12, df_exploitable, random_state=42, nb_letters = 26, kernel_in = 'rbf', kernel_pos = 'linear', C_in = 1, C_pos = 1)

# df_exploitable_100 = df_exploitable.iloc[:,1:100]

C_list = [0.1,1,2]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
n_list = [7, 12, 17]
results = []

for C_in in C_list:
    for C_pos in C_list:
        for kernel_in in kernel_list:
            for kernel_pos in kernel_list:
                for n in n_list:
                    svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(n, df_exploitable, random_state=42, nb_letters = 26, kernel_in = kernel_in, kernel_pos = kernel_pos, C_in = C_in, C_pos = C_pos)
                    results.append((C_in, C_pos, kernel_in, kernel_pos, n, accuracy_in, accuracy_pos))

results_df = pd.DataFrame(results, columns=['C_in', 'C_pos', 'kernel_in', 'kernel_pos', 'n', 'accuracy_in', 'accuracy_pos'])
results_df.to_csv('results.csv', index=False)


In [12]:
best_results = results_df.loc[(results_df['accuracy_in']*results_df['accuracy_pos']).idxmax()]
print(best_results)

C_in                 1.0
C_pos                0.1
kernel_in        sigmoid
kernel_pos        linear
n                      7
accuracy_in     0.836879
accuracy_pos    0.673913
Name: 168, dtype: object


In [8]:
svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(17, df_exploitable, random_state=42, nb_letters = 26, kernel_in = best_results['kernel_in'], kernel_pos = best_results['kernel_pos'], C_in = best_results['C_in'], C_pos = best_results['C_pos'])

# svm_model_in.predict(fsl.word_to_vector('LTDACFI').tolist())

In [9]:
# print(fsl.find_cleavage(test, threshold = 0.1))
pos_pred = np.array([fsl.find_cleavage(X,svm_model_in, svm_model_pos, threshold = 0.0, n= 17) for X in df_exploitable['P_Structure_vector'][:100]])

in_pred = 1-np.isnan(pos_pred)

pos_pred[in_pred == 0] = 0

print(accuracy_score(df_exploitable["Annotation_pos"][:100], pos_pred))

0.63
